In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import palettable
import pandas as pd
import numpy as np

from astropy import units as u
from astropy import coordinates as coords

plt.style.use("notebook.mplstyle")
from astropy.table import Table

In [2]:
import gwb

In [3]:
from groupten import read_mist_models

In [4]:
df = pd.read_csv("../data/targets.csv")
print(len(df))

In [5]:
df['gj'] = df['tgas_gmag'] - df['tmass_j']
df['gMag'] = df['tgas_gmag'] - 5*(np.log10(df['tgas_distance'])-1)

In [6]:
mist = read_mist_models.ISOCMD("../data/MIST_iso_1490194647.iso.cmd")

In [7]:
flags = pd.read_csv("../data/target_flags.csv")

In [17]:
# parsec_2MASS = Table.read("../data/parsec_2MASSJHKs.dat",
#                     format='ascii.commented_header', header_start=5)
# parsec_Gaia = Table.read("../data/parsec_Gaia.dat",
#                     format='ascii.commented_header', header_start=5)

# parsec_gj = parsec_Gaia['Gmag'] - parsec_2MASS['Jmag']
# for i, age in enumerate(ages[::10]):
#     cond = parsec_2MASS['Age']==age
#     plt.plot(parsec_gj[cond], parsec_Gaia['Gmag'][cond], '-', color=plt.cm.Blues_r(i/10.),
#              label='{:.1f}'.format(age/1e6))
# #     plt.plot(parsec_gj[cond], parsec_2MASS['Jmag'][cond], '.', color=plt.cm.plasma(i/10.))
#     plot_gjg(np.log10(age))


In [24]:
mist.abun, mist.Av_extinction

In [41]:
plt.figure(figsize=(4,5))
colors = palettable.colorbrewer.sequential.Blues_4_r.mpl_colors
handles = []
for age, color in zip([1e7, 1e8, 1e9], colors):
    iso = mist.get_age(np.log10(age))
    l, = plt.plot(iso['Gaia_G'] - iso['2MASS_J'], iso['Gaia_G'],
             label="{:.0f}".format(np.log10(age)),
             lw=1.5, zorder=-90, color=color);
    handles.append(l)


plt.scatter(df['gj'], df['gMag'],
            c=['r' if flag else 'k' for flag in flags.ir],
            edgecolors='w', linewidths=1., s=40)
plt.scatter(df['gj'].loc[flags.xray], df['gMag'].loc[flags.xray],
            edgecolors='k', linewidths=1., s=55, facecolors='None')

plt.gca().invert_yaxis()
plt.xlabel('$\mathrm{Gaia}\ G - \mathrm{2MASS}\ J$')
plt.ylabel('$M_G$')
plt.xlim(-0.5,2)
plt.ylim(10,-2)
plt.legend(handles, ['10 Myr', '100 Myr', '1 Gyr'], loc='lower left')
plt.tight_layout()
plt.savefig("../paper/figures/cmd_gjg.pdf")